<a href="https://colab.research.google.com/github/giselesousar/python-web-scraping/blob/master/webscraping_pyht_receitas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Webscraping com Python

### O que é webscraping?

Webscraping é uma técnica de extração de dados, com ela podemos coletar dados de sites. Fazemos a 'raspagem' dos dados  que são interessantes para nós.

O processo em si é bem básico, definimos os dados que queremos, escolhemos os sites, montamos o script e recebemos os dados para análise, este é o ciclo de vida do webscraping.

Exemplos: 
*   resgatar os últimos posts que foram escritos em vários blogs. 
*   o site Buscapé varre os sites que vendem os produtos pesquisados em busca dos menores preços.
*   comentários dos usuários de um produto X em 200 e-commerces diferentes para analisa-los e validar se o produto tem aceitação no mercado.

### Referencias:  

Todo o conteúdo se encontra no Github caso queira analisar o código: [ir para o repositório!](https://github.com/matheusbattisti/webscraping_python)


#Requests

Requests é uma biblioteca fazer requisições HTTP, no caso get, nas páginas que queremos extrair dados.

In [0]:
# Importando a Requests
import requests

def request_pag(link):
  # Vamos testar a biblioteca requests
  html = requests.get(link)

  print(link,end=":  ")
  print("Status Code:",html.status_code,'\n')

  # Diferente da urllib, usamos text para apresentar o conteudo que o get nos trouxe
  return html.text


#BeautifulSoup
Com a BeautifulSoup tudo será mais fácil, esta biblioteca do Python serve para extrairmos dados de HTML e XML, de forma fácil e descomplicada podemos acessar os 'nós' da estrutura do HTML da página ou até mesmo classes e pegar as informações


# Scraping dados de uma Receita

### Ingredientes


In [0]:
receita = request_pag("https://www.daninoce.com.br/receitas/croissant-caseiro-facil-de-presunto-e-queijo/")
receita_html = BeautifulSoup(receita, 'lxml')
content_html = receita_html.find_all('div', class_="content-single")

ingredientes = []
modo_preparo = []
ing = ''
m_preparo = ''
for i in content_html:
  content = i.find_all(["h3", "ul", "ol"])
  for c in range(0,len(content)):
    if(c == len(content)-1):
      modo_preparo.append(content[c])
      break
    if(str(content[c])[1] == "h" and str(content[c+1])[1] == "u"):
      ingredientes.append(content[c])
      ingredientes.append(content[c+1])
    else:
       if(str(content[c])[1] == "h" and str(content[c+1])[1] == "o"):
         modo_preparo.append(content[c])
         modo_preparo.append(content[c+1])
for i in ingredientes:
  ing += str(i)
for m in modo_preparo:
  m_preparo += str(m)
ing = ing.replace("<h3>", '').replace("</h3>", '\n').replace("<li itemprop=\"recipeIngredient\">", '').replace("</li>", '\n').replace("<ul>", '').replace("</ul>", '')
m_preparo = m_preparo.replace("<h3>", '').replace("</h3>", '\n').replace("<li itemprop=\"recipeInstructions\">", '').replace("</li>", '\n').replace("<ol>", '').replace("</ol>", '')


https://www.daninoce.com.br/receitas/croissant-caseiro-facil-de-presunto-e-queijo/:  Status Code: 200 

INGREDIENTES
10 gramas de FERMENTO BIOLÓGICO SECO
80 gramas de AÇÚCAR CRISTAL
200 ml de LEITE MORNO
2 colheres de sopa de ÓLEO
2 colheres de sopa de MANTEIGA SEM SAL
2 OVOS
1 colher de chá de SAL
750 gramas de FARINHA DE TRIGO
200 gramas de MANTEIGA SEM SAL GELADA
1 OVO para pincelar

-----------------
MODO DE PREPARO
Coloque em um recipiente o fermento biológico fresco, o açúcar cristal e o leite morno.
Misture e deixe descansar por 5 minutos.
Acrescente o óleo, a manteiga, os ovos e o sal.
Mexa bem e acrescente a farinha.
NA BATEDEIRA: coloque o gancho na batedeira e deixe na menor velocidade, misturando os ingredientes aos poucos. Quando já estiver formando uma massinha, acrescente aos poucos a manteiga gelada em pedacinhos. Aumente um pouco a velocidade da batedeira e deixe batendo por 10 minutos, até que ela fique bem macia e lisinha.
NA MÃO: misture todos os ingredientes com as

# Automatização do scrapping

In [0]:
def appendReceitas(link, links):
  receitas = link.find_all("div", class_="article-thumb")
  for r in receitas:
    links.append(r['href'])


#ENTRAR NA PAGINA INICIAL
url = "https://www.daninoce.com.br/receitas/"

pag_inicial = request_pag(url)
pag_inicial_html = BeautifulSoup(pag_inicial, 'lxml')

base_de_dados = {"receita":[],"ingredientes":[],"preparo":[],"tempo_de_preparo":[],"rendimento":[],"imagem":[]}

receitas_link = []

#pagina inicial
appendReceitas(pag_inicial_html, receitas_link)

for i in range(2, 103):
  url = "https://www.daninoce.com.br/receitas/page/"+str(i)
  pag = request_pag(url)
  pag_html = BeautifulSoup(pag, 'lxml')
  appendReceitas(pag_html, receitas_link)
for r in receitas_link:
  pag = request_pag(r)
  pag_html = BeautifulSoup(pag, 'lxml')

  #obtendo informações iniciais
  tempo = pag_html.find_all("span", class_="prepare-time")
  tempo_de_preparo = ''
  for t in tempo:
    tempo_de_preparo += str(t.text)
  rendimentos = pag_html.find_all("span", class_="rendiment")
  rendimento = ''
  for i in rendimentos:
    rendimento += str(i.text)
  titulos = pag_html.find_all("h2", class_="title title-article title-white")
  titulo = ''
  for t in titulos:
    titulo += str(t.text)
   


  #pegar receitas
  content_html = pag_html.find_all('div', class_="content-single")

  ingredientes = []
  modo_preparo = []
  ingredientes_str = ''
  preparo = ''
  for i in content_html:
    content = i.find_all(["h3", "ul", "ol"])
    for c in range(0,len(content)):
      if(c == len(content)-1):
        modo_preparo.append(content[c])
        break
      if(str(content[c])[1] == "h" and str(content[c+1])[1] == "u"):
        ingredientes.append(content[c])
        ingredientes.append(content[c+1])
      else:
        if(str(content[c])[1] == "h" and str(content[c+1])[1] == "o"):
          modo_preparo.append(content[c])
          modo_preparo.append(content[c+1])
  for ing in ingredientes:
    ingredientes_str += str(ing)
  for m in modo_preparo:
    preparo += str(m)
  ingredientes_str = ingredientes_str.replace("<h3>", '').replace("</h3>", '\n').replace("<li itemprop=\"recipeIngredient\">", '').replace("</li>", '\n').replace("<ul>", '').replace("</ul>", '')
  preparo = preparo.replace("<h3>", '').replace("</h3>", '\n').replace("<li itemprop=\"recipeInstructions\">", '').replace("</li>", '\n').replace("<ol>", '').replace("</ol>", '')

  #pegar imagem
  img = pag_html.select(".featured-image.container")
  urlImage = ''
  for j in img:
    urlImage += str(j['style']).replace("background-image: url(", '').replace(");", '').replace("'",'')

  #Adicionando na lista
  base_de_dados["receita"].append(titulo)
  base_de_dados["ingredientes"].append(ingredientes_str)
  base_de_dados["preparo"].append(preparo)
  base_de_dados["tempo_de_preparo"].append(tempo_de_preparo)
  base_de_dados["rendimento"].append(rendimento)
  base_de_dados["imagem"].append(urlImage)
print(len(base_de_dados['receita']))



https://www.daninoce.com.br/receitas/:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/2:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/3:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/4:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/5:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/6:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/7:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/8:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/9:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/10:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/11:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/12:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/13:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/14:  Status Code: 200 

https://www.daninoce.com.br/receitas/page/15:  Status Code: 200 

https://www.daninoce.com.

In [0]:
import pandas as pd

df_receitas = pd.DataFrame(data=base_de_dados)

df_receitas.to_csv (r'export_receitas_daninoce.csv', index = False, header=True)

In [0]:
df_receitas.head(5)

,receita,ingredientes,preparo,tempo_de_preparo,rendimento,imagem
0,Cachorro quente de travessa com purê de batata,Ingredientes: molho\n2 latas de MOLHO DE TOMAT...,Modo de preparo: molho\nPique a cebola em cubi...,30 minutos,10 pessoas,https://www.daninoce.com.br/wp-content/uploads...
1,Pastel de Nata ou Pastel de Belém | Receita fa...,INGREDIENTES: MASSA\n500 gramas de FARINHA\n35...,MODO DE PREPARO: MASSA\nColoque em um recipien...,120 minutos,Média de 40 porções,https://www.daninoce.com.br/wp-content/uploads...
2,Mini pavlova de chocolate marmorizadas,Ingredientes: base\n6 CLARAS\n400 gramas de AÇ...,Modo de preparo: base\nEm uma batedeira coloqu...,60 minutos,10 pessoas,https://www.daninoce.com.br/wp-content/uploads...
3,Croissant Caseiro Fácil de Presunto e Queijo,INGREDIENTES\n10 gramas de FERMENTO BIOLÓGICO ...,MODO DE PREPARO\nColoque em um recipiente o fe...,100 minutos,6 porções,https://www.daninoce.com.br/wp-content/uploads...
4,Bolo Arco-Íris: Crepe Colorido e Muito Chantilly,INGREDIENTES\n100 gramas de MANTEIGA\n3 OVOS\n...,MODO DE PREPARO\nColoque a manteiga em uma pan...,60 minutos,5 a 8 pessoas,https://www.daninoce.com.br/wp-content/uploads...
